In [23]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [111]:
MAX_PRIORITY = 2 

# PROTOTYPE

### Objectives

- Extract data from csv files to put in an appropriate data structure
- Build a network and find shortest path

In [36]:
streams = pd.read_csv("test_cases/small-streams.csv",
                      names=['PCP','StreamName','StreamType','SourceNode','DestinationNode','Size','Period','Deadline']
                     )
topology_cols = [str(i) for i in range(7)]
topology = pd.read_csv("test_cases/small-topology.csv",names=topology_cols).groupby('0')

In [37]:
streams.head(10)

,PCP,StreamName,StreamType,SourceNode,DestinationNode,Size,Period,Deadline
0,0,Flow_0,ATS,node0_0_0_0,node0_0_6_0,80,20000,12049
1,2,Flow_1,ATS,node0_0_4_1,node0_0_3_1,130,2000,13396
2,1,Flow_10,ATS,node0_0_2_1,node0_0_6_1,329,20000,12910
3,0,Flow_11,ATS,node0_0_5_1,node0_0_6_0,373,4000,19044
4,0,Flow_12,ATS,node0_0_0_0,node0_0_6_1,359,4000,17020
5,2,Flow_13,ATS,node0_0_4_1,node0_0_6_0,354,2000,17088
6,1,Flow_14,ATS,node0_0_4_1,node0_0_3_0,467,4000,16154
7,0,Flow_15,ATS,node0_0_4_1,node0_0_1_0,395,20000,15512
8,2,Flow_16,ATS,node0_0_1_1,node0_0_6_0,278,2000,14692
9,0,Flow_17,ATS,node0_0_0_0,node0_0_6_1,112,2000,15756


In [38]:
switches = topology.get_group('SW')
switches = switches.drop(columns=['4','5','6'])
switches.columns = ['DeviceType','DeviceName','Ports','Domain']

end_systems = topology.get_group('ES')
end_systems = end_systems.drop(columns=['4','5','6'])
end_systems.columns = ['DeviceType','DeviceName','Ports','Domain']

links = topology.get_group('LINK')
links.columns = ['LINK','LinkID','SourceDevice','SourcePort','DestinationDevice','DestinationPort','Domain']

In [39]:
end_systems.head()

,DeviceType,DeviceName,Ports,Domain
8,ES,node0_0_0_0,1,NaN
9,ES,node0_0_0_1,1,NaN
10,ES,node0_0_3_0,1,NaN
11,ES,node0_0_3_1,1,NaN
12,ES,node0_0_5_0,1,NaN


In [40]:
switches.head(20)

,DeviceType,DeviceName,Ports,Domain
0,SW,sw_0_0,8,NaN
1,SW,sw_0_3,8,NaN
2,SW,sw_0_5,8,NaN
3,SW,sw_0_1,8,NaN
4,SW,sw_0_4,8,NaN
5,SW,sw_0_6,8,NaN
6,SW,sw_0_2,8,NaN
7,SW,sw_0_7,8,NaN


In [41]:
links.head(20)

,LINK,LinkID,SourceDevice,SourcePort,DestinationDevice,DestinationPort,Domain
22,LINK,e1,sw_0_0,0.0,sw_0_3,0.0,NaN
23,LINK,e2,sw_0_0,1.0,sw_0_5,0.0,NaN
24,LINK,e3,sw_0_0,2.0,node0_0_0_0,1.0,NaN
25,LINK,e4,sw_0_0,3.0,node0_0_0_1,1.0,NaN
26,LINK,e5,sw_0_3,1.0,sw_0_2,0.0,NaN
27,LINK,e6,sw_0_3,2.0,sw_0_6,0.0,NaN
28,LINK,e7,sw_0_3,3.0,sw_0_7,0.0,NaN
29,LINK,e8,sw_0_3,4.0,node0_0_3_0,1.0,NaN
30,LINK,e9,sw_0_3,5.0,node0_0_3_1,1.0,NaN
31,LINK,e10,sw_0_5,1.0,sw_0_2,1.0,NaN


In [199]:
class Flow():
    def __init__(self, data_row):
        self.priority = data_row["PCP"]
        self.src = data_row["SourceNode"]
        self.dest = data_row["DestinationNode"]
        self.b = data_row["Size"]
        self.r = data_row["Size"] / data_row["Period"]
        self.deadline = data_row["Deadline"]
        self.name = data_row["StreamName"]
        self.l = data_row["Size"] # packet length
        self.transmission_delay = self.l / self.r

    def find_path(self, G: nx.MultiGraph):
        # first we find the nodes along the way
        path = nx.shortest_path(G, self.src, self.dest) 
        graph = nx.path_graph(path)
        edges = graph.edges()
        self.links = []
        for edge in edges:
            # we fill the links with information about the flows going through them
            link = G.edges.get([edge[0], edge[1], 0])
            if link["flows"].get(self.priority) == None:
                link["flows"][self.priority] = [self]
            else:
                link["flows"][self.priority].append(self)
            self.links.append({"edges": edge, "data": link})
    def get_hop_delay(self, link):
        bh = 0 # sum of burst size of higher priority streams
        rh = 0 # total reserved rate of higher priority streams
        bi = 0 # sum of burst size of interfering streams 
        hop_flows = link["data"]["flows"]
        for higher_priority in range(self.priority+1, MAX_PRIORITY+1):
            print(higher_priority)
            hp_flows = hop_flows.get(higher_priority, [])
            for flow in hp_flows:
                bh += flow.b
                rh += flow.r
        print(bh)
        print(f"r {self.r}, rh {rh}")
        print(f"trans {self.transmission_delay}")
        sp_flows = hop_flows[self.priority]
        for flow in sp_flows:
            if flow != self: # TODO unsure
                bi += flow.b


        delay = ((bh + bi) / (self.r - rh)) + self.transmission_delay
        print(f"delay {delay}")
        return delay

    def get_total_delay(self):
        total_delay = 0
        for link in self.links:
            total_delay += self.get_hop_delay(link)
        self.total_delay = total_delay



In [200]:
G = nx.MultiGraph()
# multi graph to allow multiple links between two same nodes

# this creates the links as well as the necessary nodes
for link in links.iterrows():
    source = link[1]['SourceDevice']
    destination = link[1]['DestinationDevice']
    source_port = link[1]['SourcePort']
    link_id = link[1]['LinkID']
    destination_port = link[1]['DestinationPort']
    G.add_edge(source,
               destination,
               source_port=source_port,
               destination_port=destination_port,
               link_id=link_id,
               flows=dict()
              )

for end_system in end_systems.iterrows():
    # the names are supposedly already inside the graph because of the link creation
    name = end_system[1]['DeviceName']
    G.nodes[name]['ports'] = end_system[1]['Ports']

for switch in switches.iterrows():
    name = switch[1]['DeviceName']
    G.nodes[name]['ports'] = switch[1]['Ports']

In [201]:
flows = []
for stream in streams.iterrows():
    flow = Flow(stream[1])
    flow.find_path(G)
    flows.append(flow)

In [204]:
# Create a list of all edges, and assign colors based on whether they are in the shortest path or not
edge_colors = [
    "red" if edge in path_edges or tuple(reversed(edge)) in path_edges else "black"
    for edge in G.edges()
]

# Visualize the graph
# pos = nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, node_size = 200)
# nx.draw_networkx_edges(G, pos, edge_color=edge_colors)
# nx.draw_networkx_labels(G, pos)

# plt.show()


In [205]:
flow = flows[0]

flow.get_total_delay()

1
2
0
r 0.004, rh 0
trans 20000.0
delay 207000.0
1
2
749
r 0.004, rh 0.3745
trans 20000.0
delay 12645.074224021591
1
2
2833
r 0.004, rh 0.84275
trans 20000.0
delay 12739.19523099851
1
2
802
r 0.004, rh 0.37275
trans 20000.0
delay 16813.5593220339


In [206]:
for k, item in G.edges.items():
    # print(item.get("flows"))
    print(item)

{'source_port': 0.0, 'destination_port': 0.0, 'link_id': 'e1', 'flows': {0: [<__main__.Flow object at 0x10e211280>, <__main__.Flow object at 0x10e3602c0>, <__main__.Flow object at 0x10e363d40>, <__main__.Flow object at 0x10e3610d0>, <__main__.Flow object at 0x10e3609e0>, <__main__.Flow object at 0x10e362990>, <__main__.Flow object at 0x10e360f50>, <__main__.Flow object at 0x10e363290>], 2: [<__main__.Flow object at 0x10e360740>], 1: [<__main__.Flow object at 0x10e363740>]}}
{'source_port': 1.0, 'destination_port': 0.0, 'link_id': 'e2', 'flows': {0: [<__main__.Flow object at 0x10e3602c0>, <__main__.Flow object at 0x10e360f50>, <__main__.Flow object at 0x10e363290>], 1: [<__main__.Flow object at 0x10e363740>]}}
{'source_port': 2.0, 'destination_port': 1.0, 'link_id': 'e3', 'flows': {0: [<__main__.Flow object at 0x10e211280>, <__main__.Flow object at 0x10e363d40>, <__main__.Flow object at 0x10e3610d0>, <__main__.Flow object at 0x10e362990>]}}
{'source_port': 3.0, 'destination_port': 1.0, 